In [5]:
import warnings
warnings.filterwarnings('ignore')

from TGA.utils import Dataset, GraphsizePretrained
#from tqdm import tqdm
from tqdm.notebook import tqdm
from time import time
import numpy as np

from sklearn.preprocessing import LabelEncoder
from collections import Counter
import networkx as nx

In [6]:
%%time
graph_builder = GraphsizePretrained(w=1, verbose=True,
                   pretrained_vec='/home/Documentos/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt')

400000it [00:30, 13168.37it/s]


CPU times: user 27.2 s, sys: 632 ms, total: 27.9 s
Wall time: 30.6 s


In [7]:
dataset = Dataset('/home/Documentos/datasets/classification/datasets/acm/')
fold = next(dataset.get_fold_instances(10, with_val=True))
fold._fields, len(fold.X_train)

le = LabelEncoder()
y_train = le.fit_transform(fold.y_train)
y_val = le.transform(fold.y_val)

In [8]:
graph_builder.mindf = 1
def fit(self, X, y):
    self.N = len(X)
    y_train = self.le.fit( sorted(list(set(y))) ).transform(y)
    self.n_class = len(self.le.classes_)
    
    self.class_term_freqs = Counter()
    self.term_freqs = Counter()
    self.class_freqs = Counter(y_train)

    docs = list(map(self.analyzer.build_analyzer(), self.progress_bar(X)))
    self.node_mapper = dict()
    pairs = list( zip( docs, y_train ) )
    for (doc,y) in self.progress_bar(pairs):

        #doc_in_terms = set(filter( lambda x: x in self.embeddings_dict, doc))
        doc_in_terms = set(doc)

        self.term_freqs.update( doc_in_terms )
        
        list_of_edges = list(map( lambda x: (y, x), doc_in_terms ))
        self.class_term_freqs.update( list_of_edges )
    
    self.term_freqs       = { term:v for (term,v) in self.term_freqs.items() if v >= self.mindf }    
    self.class_term_freqs = { (y,term):v for ((y,term),v) in self.class_term_freqs.items() if v >= self.mindf and term in self.term_freqs }
    self.node_mapper      = { term:self.node_mapper.setdefault(term, len(self.node_mapper)) for term in self.term_freqs.keys() }


    return self

In [9]:
fit(graph_builder,fold.X_train, fold.y_train)

100%|██████████| 19907/19907 [00:00<00:00, 27905.39it/s]


GraphsizePretrained(encoding=None,
                    pretrained_vec='/home/Documentos/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt',
                    verbose=None, w=1)

In [10]:
len(graph_builder.class_term_freqs), len(graph_builder.node_mapper)

(109333, 49334)

In [11]:
len(graph_builder.class_term_freqs), len(graph_builder.node_mapper)

(109333, 49334)

In [12]:
import torch
import dgl
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F

import networkx as nx
from dgl.nn.pytorch.conv import GraphConv, GATConv
from dgl.nn.pytorch.glob import GlobalAttentionPooling

from sklearn.preprocessing import LabelEncoder

from itertools import repeat

import torch.optim as optim
from torch.utils.data import DataLoader

Using backend: pytorch


In [13]:
def collate_train(param):
    X, y = zip(*param)
    terms_ids, docs_offsets = transform(graph_builder, X)
    return terms_ids, docs_offsets, torch.LongTensor(y)

In [57]:
def transform(self, X):
    analyzer = self.analyzer.build_analyzer()
    doc_off = [0]
    terms_idx = []
    for i,doc in enumerate(X):
        doc_in_terms = analyzer(doc)
        doc_in_terms = filter( lambda x: x in self.node_mapper, doc_in_terms )
        doc_in_terms = [ self.node_mapper[tid] for tid in doc_in_terms ]
        #doc_in_terms = set(doc_in_terms)
        doc_off.append( len(doc_in_terms) )
        terms_idx.extend( list(doc_in_terms) )
    return torch.LongTensor( terms_idx ), torch.LongTensor(doc_off)[:-1].cumsum(0)

In [64]:
class SimpleClassifier(nn.Module):
    def __init__(self, vocab_size, hidden_l, nclass, dropout=0.1, initrange = 0.5, device='cuda:0'):
        super(SimpleClassifier, self).__init__()
        
        self.doc_terms_emb = nn.EmbeddingBag(vocab_size, hidden_l, mode='mean', scale_grad_by_freq=False)
        self.norm = nn.BatchNorm1d(hidden_l).to(device)
        
        self.fc = nn.Linear(hidden_l, nclass)
        self.drop = nn.Dropout(dropout)
        
        self.initrange = initrange
        self.nclass = nclass
        
        self.init_weights()
        
        #self.labls_emb = nn.Embedding(graph_builder.n_class, 300)
    
    def forward(self, terms_idxs, docs_offsets):
        h_docs = self.doc_terms_emb( terms_idxs, docs_offsets )
        #h_docs = self.norm(h_docs)
        h_docs = self.drop(h_docs)
        pred_docs = self.fc( h_docs )
        return pred_docs

    def init_weights(self):
        self.doc_terms_emb.weight.data.uniform_(-self.initrange, self.initrange)
        
class NotTooSimpleClassifier(nn.Module):
    def __init__(self, vocab_size, hidden_l, nclass, dropout=0.1, initrange = 0.5, device='cuda:0'):
        super(NotTooSimpleClassifier, self).__init__()
        
        self.lab_terms_emb = nn.Embedding(nclass, hidden_l, scale_grad_by_freq=False)
        self.doc_terms_emb = nn.EmbeddingBag(vocab_size, hidden_l, mode='mean', scale_grad_by_freq=False)
        
        #self.fc = nn.Linear(hidden_l, nclass)
        self.fc = nn.Sequential(
            nn.Linear(hidden_l, hidden_l//2),
            nn.Sigmoid(),
            nn.Linear(hidden_l//2, nclass)
        )
        
        self.drop = nn.Dropout(dropout)
        self.norm = nn.BatchNorm1d(hidden_l).to(device)
        
        self.initrange = initrange
        self.nclass = nclass
        
        self.init_weights()
        
        #self.labls_emb = nn.Embedding(graph_builder.n_class, 300)
    
    def forward(self, terms_idxs, docs_offsets, y=None):
        h_docs = self.doc_terms_emb( terms_idxs, docs_offsets )
        h_docs = self.norm(h_docs)
        h_docs = self.drop(h_docs)
        pred_docs = self.fc( h_docs )
        
        h_labels = sc.lab_terms_emb(torch.LongTensor(range(self.nclass)).to(h_docs.device))
        
        if y is not None:
            h_labs = self.lab_terms_emb( y )
            h_labs = self.drop( h_labs )
            return h_docs, pred_docs, h_labels, h_labs
            
        return h_docs, pred_docs, h_labels

    def init_weights(self):
        self.doc_terms_emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.lab_terms_emb.weight.data.uniform_(-self.initrange, self.initrange)
        #self.fc.weight.data.uniform_(-self.initrange, self.initrange)
        #self.fc.bias.data.zero_()

In [65]:
from TGA.lossweight import cross_entropy
class NpairLoss(nn.Module):
    """the multi-class n-pair loss"""
    def __init__(self, l2_reg=0.02):
        super(NpairLoss, self).__init__()
        self.l2_reg = l2_reg

    def forward(self, anchor, target=None, positive=None):
        batch_size = anchor.size(0)
        
        if target is not None:
            target = target.view(target.size(0), 1)
            target = (target == torch.transpose(target, 0, 1)).float()
            target = target / torch.sum(target, dim=1, keepdim=True).float()
        else:
            target = torch.eye(batch_size).to(anchor.device)

        if positive is not None:
            logit = torch.matmul(anchor, torch.transpose(positive, 0, 1))
            l2_loss = torch.sum(anchor**2) / batch_size + torch.sum(positive**2) / batch_size
        else:
            logit = torch.matmul(anchor, torch.transpose(anchor, 0, 1))
            l2_loss = torch.sum(anchor**2) / batch_size
        
        loss_ce = cross_entropy(logit, target)

        loss = loss_ce + self.l2_reg*l2_loss*0.25
        return loss.float()
class SelfDistLoss(nn.Module):
    def __init__(self, l2_reg=0.02, eps = 0.00003, margin=-0.3):
        super(SelfDistLoss, self).__init__()
        self.l2_reg = l2_reg
        self.eps = eps
        self.margin = margin
        
    def forward(self, hiddens):
        L = torch.matmul(hiddens, hiddens.T)
        L = F.sigmoid(L)
        L = (L - (L.diag()-self.margin)).float()
        L = F.relu(L)
        #L = torch.exp( L )
        L = ( L > 0. ).float() * torch.exp( L )
        #L = F.normalize(L)
        
        values = L.sum(axis=1)
        svalue = max((values > 0.).sum(), self.eps)

        return values.sum()/svalue # AVG of non-zero values
class NCrossEntropyLoss(nn.Module):
    """the multi-class n-pair loss"""
    def __init__(self, k, nclass):
        super(NCrossEntropyLoss, self).__init__()
        self.ce = nn.CrossEntropyLoss()
        self.k = k
        self.nclass = nclass

    def forward(self, preds, y):
        bs = y.shape[0]

        y2 = torch.reshape(y, (bs, 1))

        idx = torch.arange(self.k)*self.nclass
        idx = torch.reshape(idx, (1,self.k)).to(y2.device)

        idx = idx+y2

        best_positive = torch.gather( preds, 1, idx )

        y2 = torch.reshape(best_positive.argmax(axis=1)*self.nclass, (bs, 1))+y2
        y2 = y2.flatten()
        
        return self.ce( preds, y2 )


In [78]:
nepochs = 2000
max_epochs = 10
drop=0.5
device = torch.device('cuda:0')
k=1
batch_size = 16

In [79]:
labls = [set()]*graph_builder.n_class
for y,term in graph_builder.class_term_freqs.keys():
    labls[y].add(term)

labls_offset = list(map(len, labls))
labls_offset = torch.LongTensor([0]+labls_offset).cumsum(0)[:-1].to(device)

terms_lab_idx = [ graph_builder.node_mapper[t] for l in labls for t in sorted(list(l)) ]
terms_lab_idx = torch.LongTensor(terms_lab_idx).to(device)

labls_offset.shape, terms_lab_idx.shape

(torch.Size([11]), torch.Size([542674]))

In [80]:
sc = SimpleClassifier(len(graph_builder.node_mapper), 300, graph_builder.n_class*k, dropout=drop).to( device )
#sc = NotTooSimpleClassifier(len(graph_builder.node_mapper), 300, graph_builder.n_class, dropout=drop).to( device )


#optimizer = optim.AdamW( sc.parameters(), lr=5e-3, weight_decay=5e-2)
optimizer = optim.AdamW( sc.parameters(), lr=5e-3, weight_decay=5e-3)
loss_func_cel = nn.CrossEntropyLoss().to( device )
#loss_func_cel = NCrossEntropyLoss(k=k, nclass=graph_builder.n_class).to( device )
loss_func_self_np = NpairLoss(l2_reg=5e-4)
loss_func_self = SelfDistLoss(l2_reg=5e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.9)



In [81]:
best = 0.
counter = 1
for e in tqdm(range(nepochs), total=nepochs):
    dl_train = DataLoader(list(zip(fold.X_train, y_train)), batch_size=batch_size,
                             shuffle=True, collate_fn=collate_train, num_workers=5)
    dl_val = DataLoader(list(zip(fold.X_val, y_val)), batch_size=batch_size,
                             shuffle=False, collate_fn=collate_train, num_workers=5)
    total_loss  = 0.
    with tqdm(total=len(y_train)+len(y_val), smoothing=0., desc=f"Epoch {e+1}") as pbar:
        total = 0
        correct  = 0
        sc.train()
        for i, (terms_idx, docs_offsets, y) in enumerate(dl_train):
            terms_idx    = terms_idx.to( device )
            docs_offsets = docs_offsets.to( device )
            y            = y.to( device )
            
            pred_docs = sc( terms_idx, docs_offsets)
            pred_docs = F.softmax(pred_docs)
            loss = loss_func_cel(pred_docs, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total      += len(y)
            y_pred      = pred_docs.argmax(axis=1)
            correct    += (y_pred == y).sum().item()
            
            toprint  = f"Train loss: {total_loss/(i+1):.4}/{loss.item():.4} "
            toprint += f'ACC: {correct/total:.4}'
            
            print(toprint, end=f"{' '*100}\r")
            
            pbar.update( len(y) )
            
        scheduler.step()
        total = 0
        correct  = 0
        sc.eval()
        print()
        for i, (terms_idx, docs_offsets, y) in enumerate(dl_val):
            terms_idx    = terms_idx.to( device )
            docs_offsets = docs_offsets.to( device )
            y            = y.to( device )
            
            pred_docs = sc( terms_idx, docs_offsets )
            pred_docs = F.softmax(pred_docs)
            
            y_pred      = pred_docs.argmax(axis=1)
            correct    += (y_pred == y).sum().item()
            total      += len(y)
            
            print(f'Val ACC: {correct/total:.4}/{best:.4}', end=f"{' '*100}\r")
            
            pbar.update( len(y) )
        if (correct/total) > best:
            best = (correct/total)
            counter = 1
            print(f'New Best Val ACC: {best:.4}')
        elif counter > max_epochs:
            print()
            print(f'Best Val ACC: {best:.4}')
            break
        else:
            counter += 1

Train loss: 1.957/1.814 ACC: 0.6187                                                                                                    
New Best Val ACC: 0.7435                                                                                               



Train loss: 1.726/1.749 ACC: 0.8286                                                                                                    
New Best Val ACC: 0.7679                                                                                                  



Train loss: 1.656/1.543 ACC: 0.8932                                                                                                    
Val ACC: 0.7659/0.7679                                                                                                    


Train loss: 1.626/1.543 ACC: 0.9207                                                                                                    
Val ACC: 0.7615/0.7679                                                                                                    


Train loss: 1.61/1.852 ACC: 0.9363                                                                                                     
Val ACC: 0.7571/0.7679                                                                                                    


...

Train loss: 1.581/1.872 ACC: 0.9636                                                                                                    
Val ACC: 0.7547/0.7679                                                                                                    


Train loss: 1.578/1.543 ACC: 0.9654                                                                                                    
Val ACC: 0.7579/0.7679                                                                                                    


Train loss: 1.576/1.543 ACC: 0.9676                                                                                                    
Val ACC: 0.7507/0.7679                                                                                                    


Train loss: 1.574/1.543 ACC: 0.969                                                                                                     
Val ACC: 0.7503/0.7679                                                                                                    


Train loss: 1.573/1.543 ACC: 0.9705                                                                                                    
Val ACC: 0.7491/0.7679                                                                                                    
Best Val ACC: 0.7679




In [ ]:
# Best Val ACC: 0.7667



# bs = 64
# ACM    Train loss: 3.598 Ls1: 0.004926, Ls2: 3.013, Ls3: 0.5797 ACC: 0.8418                                                                                                    
#        Val ACC: 0.7727  / 79.92
# simple:
# Best Val ACC: 0.7804


# agnews Train loss: 4.716 Ls1: 0.005742, Ls2: 4.531, Ls3: 0.18 ACC: 0.94                                                                                                        
#        Val ACC: 0.9195  / 94.04
# simple:
# Best Val ACC: 0.9222


# 20ng   Train loss: 2.331 Ls1: 0.02, Ls2: 2.28, Ls3: 0.0286 ACC: 0.9958                                                                                                      
#        Val ACC: 0.8948    / 92.65 (dropout=0.5)
# simple:
# Best Val ACC: 0.8869


# reut   Val ACC: 0.7185 / 79.11
#         Train loss: 4.026 Ls1: 0.00845, Ls2: 3.15, Ls3: 0.863 ACC: 0.7628                                                                                                    
#         Val ACC: 0.7081
# simple:
# Best Val ACC: 0.6926


# webkb  Train loss: 3.794 Ls1: 0.00743, Ls2: 3.66, Ls3: 0.13 ACC: 0.9705                                                                                                     
#        Val ACC: 0.7436 
# simple:
# Best Val ACC: 0.7521

In [18]:
[pred_docs]*10

[tensor([[1.0632e-04, 3.7715e-04, 7.5147e-05, 2.6979e-04, 1.6213e-05, 5.8491e-05,
          3.1132e-05, 9.9839e-01, 8.4278e-05, 1.0278e-04, 4.8512e-04],
         [2.5973e-05, 9.2950e-06, 1.1893e-05, 3.5760e-07, 2.2005e-06, 1.2285e-05,
          4.2648e-06, 9.9907e-01, 8.4985e-04, 1.3562e-05, 3.4813e-06],
         [1.5521e-04, 7.8153e-05, 3.1194e-05, 9.9375e-01, 2.7410e-05, 8.4211e-05,
          3.7413e-04, 9.6144e-04, 1.9809e-03, 7.0491e-05, 2.4866e-03],
         [3.6640e-06, 1.5031e-06, 3.9975e-07, 8.4714e-08, 6.3422e-08, 9.9999e-01,
          8.6444e-07, 1.4853e-09, 1.6707e-08, 3.1322e-06, 1.9504e-09],
         [5.9767e-06, 4.3367e-06, 1.9636e-05, 2.2277e-07, 6.6186e-07, 9.4857e-06,
          2.7996e-05, 1.4547e-03, 9.9847e-01, 4.2663e-06, 6.0288e-06],
         [5.4184e-07, 1.1613e-06, 5.9193e-06, 5.6562e-04, 2.5102e-07, 1.1155e-06,
          4.9945e-08, 1.1210e-05, 9.9941e-01, 4.8868e-07, 2.1379e-07],
         [9.8767e-04, 1.2335e-03, 2.6287e-03, 7.7616e-01, 3.0276e-04, 1.1513e-03,


In [19]:
self_k=2
self_nclass=graph_builder.n_class
batch_size = y.shape[0]

kpreds = torch.cat([pred_docs]*self_k, axis=1).to('cpu:0')
y2 = torch.reshape(y, (batch_size, 1)).to('cpu:0')

idx = torch.arange(self_k)*self_nclass
idx = torch.reshape(idx, (1,self_k)).to('cpu:0')

idx = idx+y2

best_positive = torch.gather( kpreds, 1, idx )

y2 = torch.reshape(best_positive.argmax(axis=1)*self_nclass, (batch_size, 1))+y2
y2 = y2.flatten()

y2,y

(tensor([19, 18, 14, 16, 19, 19, 14, 18, 13, 19, 14, 16, 13, 18, 14, 16, 20, 19,
         21, 14, 12, 14, 14, 18, 14, 18, 11, 19, 18, 19, 21, 18, 19, 13, 21, 21,
         21, 13, 19, 19, 13, 12, 14, 21, 14, 19, 19, 18, 14, 14, 11, 21, 11, 14,
         18, 19, 12, 21, 21, 19, 16, 12, 14]),
 tensor([ 8,  7,  3,  5,  8,  8,  3,  7,  2,  8,  3,  5,  2,  7,  3,  5,  9,  8,
         10,  3,  1,  3,  3,  7,  3,  7,  0,  8,  7,  8, 10,  7,  8,  2, 10, 10,
         10,  2,  8,  8,  2,  1,  3, 10,  3,  8,  8,  7,  3,  3,  0, 10,  0,  3,
          7,  8,  1, 10, 10,  8,  5,  1,  3], device='cuda:0'))

In [20]:
y

tensor([ 8,  7,  3,  5,  8,  8,  3,  7,  2,  8,  3,  5,  2,  7,  3,  5,  9,  8,
        10,  3,  1,  3,  3,  7,  3,  7,  0,  8,  7,  8, 10,  7,  8,  2, 10, 10,
        10,  2,  8,  8,  2,  1,  3, 10,  3,  8,  8,  7,  3,  3,  0, 10,  0,  3,
         7,  8,  1, 10, 10,  8,  5,  1,  3], device='cuda:0')

In [21]:
kpreds[0][idx[0]].shape

torch.Size([2])

In [22]:
# corretos                 [ [1,0,0,1,0,0,1,0,0], [0,0,1,0,0,1,0,0,1], [0,0,1,0,0,1,0,0,1], [0,1,0,0,1,0,0,1,0] ]
preds_teste = torch.Tensor([ [1,0,0,0,0,0,0,0,0], [1,0,0,0,0,0,0,0,0], [0,0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,0,1] ])
y_teste     = torch.LongTensor([ [0, 3, 6], [2, 5, 8], [2, 5, 8], [1,4,7] ])
print(preds_teste[0][y_teste[0]])
print(preds_teste[1][y_teste[1]])
print(preds_teste[2][y_teste[2]])
print(preds_teste[3][y_teste[3]])
preds_teste.shape, y_teste.shape

tensor([1., 0., 0.])
tensor([0., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 0.])


(torch.Size([4, 9]), torch.Size([4, 3]))

In [23]:
help( torch.index_select )

Help on built-in function index_select:

index_select(...)
    index_select(input, dim, index, out=None) -> Tensor
    
    Returns a new tensor which indexes the :attr:`input` tensor along dimension
    :attr:`dim` using the entries in :attr:`index` which is a `LongTensor`.
    
    The returned tensor has the same number of dimensions as the original tensor
    (:attr:`input`).  The :attr:`dim`\ th dimension has the same size as the length
    of :attr:`index`; other dimensions have the same size as in the original tensor.
    
    .. note:: The returned tensor does **not** use the same storage as the original
              tensor.  If :attr:`out` has a different shape than expected, we
              silently change it to the correct shape, reallocating the underlying
              storage if necessary.
    
    Args:
        input (Tensor): the input tensor.
        dim (int): the dimension in which we index
        index (LongTensor): the 1-D tensor containing the indices to index


In [24]:
F.softmax(torch.matmul( h_labels, h_labels.T ))

NameError: name 'h_labels' is not defined

In [ ]:
labls_offset